## Trabalho de IA - Previsão de Churn e Segmentação de Clientes em Telecom

### 4.1 Etapa 1: Coleta, Limpeza e EDA
1. Importar o CSV no Pandas, verificar estatísticas básicas (.describe(), .info()).

In [ ]:
# 1. Importar Pandas e Numpy e em seguida importar o CSV no Pandas

import pandas as pd

try:
    df = pd.read_csv("../data/WA_Fn-UseC_-Telco-Customer-Churn.csv")
except Exception as e:
    print(f"Erro ao importar o dataset: {e}")
    exit()

df.head()

In [ ]:
# Verificar estatísticas
print("\n Informações do DataFrame (.info()) ")
df.info()

In [ ]:
print("\n Estatísticas Descritivas (.describe()) ")
# Incluir colunas de objetos para uma visão mais completa
df.describe(include='all')

 2. Tratar valores faltantes e inconsistências

In [ ]:
# A coluna 'TotalCharges' é do tipo 'object'(string) mas deveria ser numérica.
# Pode haver espaços em branco que precisam ser tratados e convertidos para numérico.

print("\n Verificando valores únicos em colunas categóricas ")
for column in df.columns:
    if df[column].dtype == 'object':
        print(f"\nColuna '{column}':")
        print(df[column].value_counts())

In [ ]:
# Tratamento da coluna 'TotalCharges'
# Convertendo 'TotalCharges' para numérico, tratando erros com 'coerce' (transforma em NaN)
print("\n Tratando a coluna 'TotalCharges' ")
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

print(df['TotalCharges'])

In [ ]:
# Preencher valores faltantes na coluna 'TotalCharges'
# Para esses caso, vamos preencher com 0, pois representa novos clientes.
df['TotalCharges'].fillna(0, inplace=True)

In [ ]:
# Verificação final de valores faltantes
print("\n Verificação final de valores faltantes ")
print(df.isnull().sum())

Tratamento de outras inconsistências (se houver, baseado nas value_counts)
Exemplo: Se 'No phone service' ou 'No internet service' fossem tratados de forma diferente ou se houvesse algum erro de digitação. No dataset Telco Churn, essas são categorias válidas.

In [ ]:
# Convertendo a coluna 'Churn' para valores binários (0 e 1)
print("\n Convertendo 'Churn' para valores binários (0 e 1) ")
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
print(df['Churn'].value_counts())

In [ ]:
print("\n DataFrame após tratamento ")
df.info()
print(df.describe(include='all'))
df.head()

3.  Converter variáveis categóricas via One-Hot Encoding

In [ ]:
print("\n Realizando One-Hot Encoding em variáveis categóricas ")

# Identificar colunas categóricas para OneHot Encoding
# Excluímos 'customerID' porque é um identificador e 'Churn' porque já foi convertida para numérica.
categorical_cols = [col for col in df.columns if df[col].dtype == 'object' and col != 'customerID']

In [ ]:
# Aplicar One-Hot Encoding usando pd.get_dummies
# 'drop_first=True' é usado para evitar a armadilha da variável dummy, removendo a primeira categoria de cada coluna.
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)


In [ ]:
#Após a utilização do OneHot Enconding, pegaremos as colunas do tipo booleano e convertemos e converteremos para colunas numéricas.

print("\n Convertendo colunas booleanas (True/False) para 0 e 1 ")
for column in df.columns:
    if df[column].dtype == bool:
        df[column] = df[column].astype(int)
        print(f"Coluna '{column}' convertida de booleano para int (0/1).")

print("\n DataFrame após OneHot Encoding ")
df.head()

In [ ]:
df.info()
print(df.describe(include='all'))

4. Escalonar variáveis numéricas com StandardScaler

In [ ]:
print("\n Escalonando variáveis numéricas com StandardScaler ")

# Identificar colunas numéricas para escalonar
# Excluímos 'customerID' (identificador) e 'Churn' (variável alvo).
# Colunas que já são binárias (0/1) após o One-Hot Encoding não precisam estritamente de escalonamento,
# mas o StandardScaler as tratará, resultando em valores -1 ou 1 (aproximadamente) se houver apenas duas categorias,
# ou 0 se forem todas iguais.
numerical_cols = [col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col not in ['customerID', 'Churn']]

In [ ]:
from sklearn.preprocessing import StandardScaler

# Inicializar o StandardScaler
scaler = StandardScaler()

In [ ]:
# Aplicar o escalonamento nas colunas numéricas selecionadas
# O fit_transform retorna um array NumPy
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [ ]:
print("\n DataFrame após Escalonamento com StandardScaler ")
df.info()
print(df.describe(include='all'))
df.head()

5. Gerar gráficos exploratórios (histogramas, boxplots, matrizes de correlação)

In [ ]:
# Para esse passo iremos importar 2 bibliotecas que irão nos auxiliar com a geração dos graficos
# As bibliotecas são matplotlib e seaborn, enquanto a matplotlib faz a geração de graficos
# A seaborn deixa eles com uma aparencia melhor.

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Calcular o número de linhas e colunas para os subplots dinamicamente
num_numerical_cols = len(numerical_cols)

n_cols = 3  # Número de colunas fixo por linha nos gráficos

n_rows = num_numerical_cols // n_cols
if num_numerical_cols % n_cols != 0:
    n_rows += 1

Gerando Histogramas para variáveis numéricas

In [ ]:
plt.figure(figsize=(n_cols * 5, n_rows * 4))
for i, col in enumerate(numerical_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df[col], kde=True)
    plt.title(f'Histograma de {col}')
    plt.xlabel(col)
    plt.ylabel('Frequência')
plt.tight_layout()
plt.show()

Boxplot para variáveis numéricas (para identificar outliers e distribuição)

In [ ]:
print("\n Gerando Boxplots para variáveis numéricas...")
plt.figure(figsize=(n_cols * 5, n_rows * 4))
for i, col in enumerate(numerical_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(y=df[col])
    plt.title(f'Boxplot de {col}')
    plt.ylabel(col)
plt.tight_layout()
plt.show()

Matriz de Correlação

In [ ]:
# Calculando a matriz de correlação do DataFrame inteiro (já processado)
print("\n Gerando Matriz de Correlação...")
plt.figure(figsize=(20, 15))
correlation_matrix = df.drop(columns=['customerID']).corr() # Excluímos 'customerID' da correlação
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matriz de Correlação entre as Variáveis')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()